In [21]:
import pandas as pd
import sqlite3

In [2]:
conn = sqlite3.connect('NBAdata.db')

## 1. Quem são os maiores pontuadores da história: tanto em pontos totais quanto por jogo?

In [28]:
query = """
            SELECT namePlayer,
            COUNT(DISTINCT slugSeason) as Temporadas,
            COUNT(*) as Jogos,
            SUM(pts) as TotalPontos,
            AVG(pts) as PorJogo 
            FROM PlayersBoxscore
            GROUP BY namePlayer
            ORDER BY PorJogo DESC
            LIMIT 20;
"""

df_read = pd.read_sql(query, con=conn)
df_read

,namePlayer,Temporadas,Jogos,TotalPontos,PorJogo
0,Michael Jordan,15,1251,38279,30.598721
1,Wilt Chamberlain,14,1205,35024,29.065560
2,Kevin Durant,13,1035,28337,27.378744
3,Jerry West,14,1085,29652,27.329032
4,Elgin Baylor,14,981,26807,27.326198
5,LeBron James,18,1576,42998,27.282995
6,Allen Iverson,14,985,26479,26.882234
7,Bob Pettit,11,883,23171,26.241223
8,George Gervin,10,850,22300,26.235294
9,Luka Doncic,3,212,5551,26.183962


## 2. Quem são os melhores arremessadores - considerando 2FG, 3FG, FT - da história?

As métricas 2FG e 3FG podem ser melhor avaliadas em uma nova chamada eFG= (FGM + (0.5*3FGM)) / FGA.
Também é necessária a obtenção das métricas de desempenho derivadas das feitas (2FGM, 3FGM, FTM) com as tentadas (2FGA, 3FGA, FTA).

In [55]:
query = """
            SELECT namePlayer,
            COUNT(*) as Jogos,
            SUM(fg2m)/SUM(fg2a) as FG2,
            SUM(fg3m)/SUM(fg3a) as FG3,
            SUM(ftm)/SUM(fta) as FT,
            (SUM(fgm) + 0.5*SUM(fg3m))/sum(fga) as eFG
            FROM PlayersBoxscore
            WHERE (fg2a IS NOT NULL AND fg3a IS NOT NULL AND fta IS NOT NULL AND fga IS NOT NULL)
            GROUP BY namePlayer
            HAVING Jogos >= 500
            ORDER BY eFG DESC
            LIMIT 20;
"""

df_read = pd.read_sql(query, con=conn)
df_read

,namePlayer,Jogos,FG2,FG3,FT,eFG
0,DeAndre Jordan,989,0.674629,0.076923,0.467725,0.673316
1,Rudy Gobert,588,0.648121,0.000000,0.626449,0.646858
2,Artis Gilmore,693,0.625474,0.076923,0.717536,0.624383
3,Andris Biedrins,530,0.596972,0.000000,0.501256,0.596484
4,Tyson Chandler,1236,0.595964,0.000000,0.643438,0.594925
5,Steven Adams,647,0.595327,0.062500,0.546875,0.593459
6,Dwight Howard,1307,0.589750,0.157895,0.563846,0.587161
7,Joe Ingles,590,0.524272,0.412188,0.764706,0.585088
8,Amir Johnson,924,0.591108,0.326284,0.671818,0.583904
9,Stephen Curry,874,0.517655,0.428219,0.906757,0.579794


## 3. Quem são os melhores pontuadores de 3 pontos da história? Qual sua eficiência?

In [80]:
query = """
            SELECT namePlayer,
            SUM(fg3m) as Cestas3Pts,
            SUM(fg3m)/SUM(fg3a) as FG3
            FROM PlayersBoxscore
            GROUP BY namePlayer
            HAVING Cestas3Pts >= 200
            ORDER BY Cestas3Pts DESC
            LIMIT 20;
"""

df_read = pd.read_sql(query, con=conn)
df_read

,namePlayer,Cestas3Pts,FG3
0,Ray Allen,3358.0,0.400334
1,Stephen Curry,3302.0,0.428219
2,Reggie Miller,2880.0,0.394197
3,James Harden,2786.0,0.359298
4,Kyle Korver,2704.0,0.424890
5,Jason Terry,2503.0,0.380164
6,Paul Pierce,2419.0,0.366904
7,LeBron James,2411.0,0.343398
8,Vince Carter,2409.0,0.369479
9,Jamal Crawford,2328.0,0.346068


## 4. Quais são as equipes com maior proporção de vitórias na temporada regular e nos playoffs?

In [97]:
query = """
            SELECT slugTeam as Equipe,typeSeason as Periodo,
            COUNT(*) as Partidas,
            SUM(CASE WHEN outcomeGame = 'W' THEN 1 ELSE 0 END) as Vitorias,
            SUM(CASE WHEN outcomeGame = 'W' THEN 1 ELSE 0 END) * 100 / COUNT(*) as Aproveitamento
            FROM PlayersBoxscore
            GROUP BY Equipe,typeSeason
            HAVING Partidas >= 500
            ORDER BY Aproveitamento DESC, Partidas DESC
            LIMIT 30;
"""

df_read = pd.read_sql(query, con=conn)
df_read

,Equipe,Periodo,Partidas,Vitorias,Aproveitamento
0,GSW,Playoffs,1508,1029,68
1,SAS,Regular Season,21741,14585,67
2,CHS,Regular Season,2382,1458,61
3,MNL,Playoffs,978,604,61
4,LAL,Regular Season,48824,29586,60
5,SAS,Playoffs,3085,1861,60
6,OKC,Regular Season,10762,6395,59
7,LAL,Playoffs,6275,3748,59
8,BOS,Regular Season,59276,34929,58
9,AND,Regular Season,620,365,58


## 5. Quais equipes são as mais carrascas dos grandes jogadores nos playoffs? Esses jogadores são carrascos de quais equipes?

In [124]:
# Equipes que mais deram trabalho para os grandes jogadores
query = """
        WITH added_row_number AS(    
            SELECT namePlayer as Jogador,slugOpponent as Adversario,
            COUNT(*) as Partidas,
            SUM(CASE WHEN outcomeGame = 'L' THEN 1 ELSE 0 END) as Derrotas,
            SUM(CASE WHEN outcomeGame = 'W' THEN 1 ELSE 0 END) * 100 / COUNT(*) as Aproveitamento,
            ROW_NUMBER() OVER(PARTITION BY namePlayer ORDER BY SUM(CASE WHEN outcomeGame = 'W' THEN 1 ELSE 0 END) * 100 / COUNT(*) ASC) AS row_number
            FROM PlayersBoxscore
            WHERE (typeSeason = 'Playoffs' AND 
            namePlayer IN ('Michael Jordan','LeBron James','Magic Johnson','Kareem Abdul-Jabbar',
            'Kobe Bryant','Stephen Curry','Larry Bird','Wilt Chamberlain','Bill Russell','Tim Duncan','Hakeem Olajuwon',
            'Oscar Robertson','Jerry West','Patrick Ewing','Karl Malone','David Robinson', "Shaquille O'Neal"))
            GROUP BY namePlayer, slugOpponent
            HAVING Partidas >= 10)
        SELECT * FROM added_row_number
        WHERE row_number = 1
        ORDER BY Aproveitamento ASC;
"""

df_read = pd.read_sql(query, con=conn)
df_read

,Jogador,Adversario,Partidas,Derrotas,Aproveitamento,row_number
0,David Robinson,UTH,10,7,30,1
1,LeBron James,GSW,22,15,31,1
2,Karl Malone,CHI,12,8,33,1
3,Hakeem Olajuwon,SEA,28,18,35,1
4,Patrick Ewing,CHI,34,22,35,1
5,Jerry West,BOS,38,24,36,1
6,Kareem Abdul-Jabbar,DET,11,7,36,1
7,Oscar Robertson,BOS,24,15,37,1
8,Magic Johnson,DET,10,6,40,1
9,Wilt Chamberlain,BOS,49,29,40,1


In [125]:
# Equipes que mais sofreram na mão do jogador
query = """
        WITH added_row_number AS(    
            SELECT namePlayer as Jogador,slugOpponent as Adversario,
            COUNT(*) as Partidas,
            SUM(CASE WHEN outcomeGame = 'L' THEN 1 ELSE 0 END) as Derrotas,
            SUM(CASE WHEN outcomeGame = 'W' THEN 1 ELSE 0 END) * 100 / COUNT(*) as Aproveitamento,
            ROW_NUMBER() OVER(PARTITION BY namePlayer ORDER BY SUM(CASE WHEN outcomeGame = 'W' THEN 1 ELSE 0 END) * 100 / COUNT(*) DESC) AS row_number
            FROM PlayersBoxscore
            WHERE (typeSeason = 'Playoffs' AND 
            namePlayer IN ('Michael Jordan','LeBron James','Magic Johnson','Kareem Abdul-Jabbar',
            'Kobe Bryant','Stephen Curry','Larry Bird','Wilt Chamberlain','Bill Russell','Tim Duncan','Hakeem Olajuwon',
            'Oscar Robertson','Jerry West','Patrick Ewing','Karl Malone','David Robinson', "Shaquille O'Neal"))
            GROUP BY namePlayer, slugOpponent
            HAVING Partidas >= 10)
        SELECT * FROM added_row_number
        WHERE row_number = 1
        ORDER BY Aproveitamento DESC;
"""

df_read = pd.read_sql(query, con=conn)
df_read

,Jogador,Adversario,Partidas,Derrotas,Aproveitamento,row_number
0,Larry Bird,CHI,10,0,100,1
1,LeBron James,ATL,12,0,100,1
2,Stephen Curry,POR,10,0,100,1
3,Magic Johnson,SEA,13,1,92,1
4,Shaquille O'Neal,NJN,13,1,92,1
5,Michael Jordan,MIA,11,1,90,1
6,Kareem Abdul-Jabbar,SAN,16,2,87,1
7,Tim Duncan,DEN,10,2,80,1
8,Kobe Bryant,POR,21,5,76,1
9,Jerry West,CHI,16,4,75,1
